In [1]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [9]:
#I don't think this is still needed
for obj in decode_stacked(fc):
    ind = 0
    try: versions = list(obj['versions'].keys())
    except: continue
    try: release_dates = obj['time']
    except: continue
    info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
    for key in versions:
        try:
            info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
            if pkg in info[key]['dependencies']: ind = 1
        except: continue
    if ind :
        dependent[pkg].append({obj['name']: info})

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [ ]:
#user-info
#    pji
#        version 1.0
#        date
#        
#        version 1.1
#        date
#    gifnoc
#        version
#        date
        

In [28]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [29]:
for k in dependent.keys():
    for i in dependent[k]:
        print(i)
        #print(k, dependent[k][:20])
        #print("len = ", len(dependent[k]))
        #break

jsonscript-proxy
senasaikou-yeoman
webpack-egoist
alex-d.js
advanced-image-loader
webpack-addons-ylvis
primeng-custom
jicli
@lgeiger/electron-builder
@touched/map-tool-project
generator-arc
node-private-tools
fhir2
webpack-ie8
mocoolka-tools
misstep
@cysonius/schema-utils
ng2-dnd-kokatsuna
qweb-client
@johnf/electron-builder
qor
json-framework
hy-checkbox
babel-plugin-react-native-css
webpack-universal
babel-plugin-react-css-modules-avion
general-req
@ngxvoice/ngx-voicelistner
electron-builder-admin
iqm
freemamba
isomorphic-webpack
webpack-tal
search-list-react
sea-floor
carousel-react
nicholas_util
search-input-react
json-injector
es-webpack
@frctl/support
ajv-interface-keywords
stygian
lrbceshi
gh-lint
@jose_santacruz/middy
@ddder/webpack
vbot
outils-ren
canvas-fingerprint
firebase-forum
firebase-feedback
egeria-mercurius
webpack-no-postintall-fail
webpack-no-postinstall-fail
calendar-source-provider
egg-ajv-keywords
eslint-plugin-aurelia-event-aggregator
appache
comanche
webpack_yb


In [ ]:
#ajv-keywords
    #jsconscript-proxy
    #senasaikou-yeoman
    #webpack-egoist
#user-info
    #ajv-keywords
        

In [18]:
for k in dependent.keys():
    print(dependent[k][:20])
    print("len = ", len(dependent[k]))
    break

[{'jsonscript-proxy'}, {'senasaikou-yeoman'}, {'webpack-egoist'}, {'alex-d.js'}, {'advanced-image-loader'}, {'webpack-addons-ylvis'}, {'primeng-custom'}, {'jicli'}, {'@lgeiger/electron-builder'}, {'@touched/map-tool-project'}, {'generator-arc'}, {'node-private-tools'}, {'fhir2'}, {'webpack-ie8'}, {'mocoolka-tools'}, {'misstep'}, {'@cysonius/schema-utils'}, {'ng2-dnd-kokatsuna'}, {'qweb-client'}, {'@johnf/electron-builder'}]
len =  157


In [17]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=7)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    print(ent)

1
{'dl': [{'end': '2015-03-08', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-16', 'downloads': 0, 'start': '2015-03-09'}, {'end': '2015-03-24', 'downloads': 0, 'start': '2015-03-17'}, {'end': '2015-04-01', 'downloads': 0, 'start': '2015-03-25'}, {'end': '2015-04-09', 'downloads': 0, 'start': '2015-04-02'}, {'end': '2015-04-17', 'downloads': 0, 'start': '2015-04-10'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-18'}, {'end': '2015-05-03', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-11', 'downloads': 0, 'start': '2015-05-04'}, {'end': '2015-05-19', 'downloads': 0, 'start': '2015-05-12'}, {'end': '2015-05-27', 'downloads': 0, 'start': '2015-05-20'}, {'end': '2015-06-04', 'downloads': 0, 'start': '2015-05-28'}, {'end': '2015-06-12', 'downloads': 0, 'start': '2015-06-05'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-13'}, {'end': '2015-06-28', 'downloads': 0, 'start': '2015-06-21'}, {'end': '2015-07-06', 'downloads': 0, 'start': '2015-06-29'}

In [12]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for k in ent.keys():
    for k2 in ent[k]:
        download_list.append(k2["downloads"])
for k1 in ent.keys():
    for k3 in ent[k1]:
        start_date.append(k3["start"])
#map, reduce, filter, lambda
log_list = [math.log(x+1) for x in download_list]
difference_list = []
for i in range(1,len(log_list)-1):
    dif = log_list[i] - log_list[i-1]
    difference_list.append( (log_list[i] - log_list[i-1]))
    if dif > 2:
        print("Start date of spike: " +start_date[i])
    if dif < -1.5:
        print("Start date of fall: " +start_date[i])
print(difference_list)
#print(log_list)
#print(download_list)


Start date of spike: 2016-05-30
Start date of fall: 2016-07-09
Start date of spike: 2016-09-11
Start date of spike: 2016-09-19
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.31748811353631, -0.09798040836020316, -1.0006318803079064, -0.4462871026284194, 0.8649974374866045, -2.028148247292285, 0.7884573603642704, -1.01160091167848, 0.4054651081081644, 1.5404450409471488, -0.6931471805599454, -0.3364722366212125, 0.40546510810816416, 7.976114524479225, 2.7171825042035262, 0.01668275863397284, -0.08272209347757276, 0.15160414714627457, -0.03157683305186332, -0.03669678476164506, -0.015437847201415167, 0.24452899573659792, -0.0746371329039448, 0.18857948177220507, 0.029508033162995773, -0.1288194108699816, -0.469719520755401, 0.40323731150070685, 0.303466809

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=7)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

1


In [19]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    temp = r['dl']
    print(temp[0])
    break

{'end': '2015-03-08', 'downloads': 0, 'start': '2015-03-01'}
